In [1]:
from casatasks import concat
import glob, os

In [2]:
shifted = sorted(glob.glob("/data/jfaber/dsa110-contimg/J1459_2025-05-29/msfiles/base/*.ms"))

In [3]:
from casatools import table
def missing_tel_loc(msname):
    tb = table(); tb.open(f'{msname}/OBSERVATION'); ok = 'TELESCOPE_LOCATION' in tb.colnames(); tb.close()
    return not ok

for ms in shifted:                               # scans = list of *_pc.ms
    if missing_tel_loc(ms):
        print(ms, 'needs TELESCOPE_LOCATION')


/data/jfaber/dsa110-contimg/J1459_2025-05-29/msfiles/base/J1459_250529_60min_centered.ms needs TELESCOPE_LOCATION


In [4]:
# --- define the target phase centre (J1459+716) -------------------
pc_ra  = "14h59m07.583867s"
pc_dec = "71d40m19.867740s"
new_pc = f"J2000 {pc_ra} {pc_dec}"


In [ ]:
from casatasks import phaseshift
from casatasks import mstransform
import glob, os

short_scans = sorted(glob.glob("/data/jfaber/dsa110-contimg/J1459_2025-05-29/msfiles/base/*.ms"))
shifted     = []

for vis in short_scans:
    out = vis.replace(".ms", "_pc.ms")
    
    mstransform(vis=vis, outputvis=out, datacolumn='all',
            phasecenter=new_pc, regridms=False)  # set regridms as needed

    shifted.append(out)


In [25]:
import casatools

def check_phase_center(ms_file):
    """
    Check the phase center information in an MS file
    """
    ms = casatools.ms()
    tb = casatools.table()
    
    try:
        # Open the FIELD table
        tb.open(f'{ms_file}/FIELD')
        
        # Get phase center information
        phase_dir = tb.getcol('PHASE_DIR')
        field_names = tb.getcol('NAME')
        
        print(f"MS: {ms_file}")
        print("Field information:")
        for i, name in enumerate(field_names):
            ra_rad = phase_dir[0, 0, i]
            dec_rad = phase_dir[1, 0, i]
            
            # Convert to degrees
            ra_deg = np.rad2deg(ra_rad)
            dec_deg = np.rad2deg(dec_rad)
            
            # Convert to HMS/DMS
            from astropy.coordinates import SkyCoord
            from astropy import units as u
            coord = SkyCoord(ra_rad * u.rad, dec_rad * u.rad, frame='icrs')
            
            print(f"  Field {i} ({name}):")
            print(f"    RA:  {ra_deg:.6f}° = {coord.ra.to_string(u.hour)}")
            print(f"    Dec: {dec_deg:.6f}° = {coord.dec.to_string(u.deg)}")
        
    finally:
        tb.close()

# Usage:
check_phase_center("/data/jfaber/dsa110-contimg/J1459_2025-05-29/msfiles/base/J1459_250529_60min.ms")  # Before
check_phase_center("/data/jfaber/dsa110-contimg/J1459_2025-05-29/msfiles/base/J1459_250529_60min_centered.ms")  # After

MS: /data/jfaber/dsa110-contimg/J1459_2025-05-29/msfiles/base/J1459_250529_60min.ms
Field information:
  Field 0 (drift_ra14h27m07s):
    RA:  216.777442° = 14h27m06.58603501s
    Dec: 71.745696° = 71d44m44.50640592s
  Field 1 (drift_ra14h27m20s):
    RA:  216.831593° = 14h27m19.58235933s
    Dec: 71.745622° = 71d44m44.23991006s
  Field 2 (drift_ra14h27m33s):
    RA:  216.885744° = 14h27m32.57863269s
    Dec: 71.745548° = 71d44m43.97305706s
  Field 3 (drift_ra14h27m46s):
    RA:  216.939895° = 14h27m45.57485501s
    Dec: 71.745474° = 71d44m43.70584718s
  Field 4 (drift_ra14h27m59s):
    RA:  216.994046° = 14h27m58.57102624s
    Dec: 71.745400° = 71d44m43.43828065s
  Field 5 (drift_ra14h28m12s):
    RA:  217.048196° = 14h28m11.5671463s
    Dec: 71.745325° = 71d44m43.17035772s
  Field 6 (drift_ra14h28m25s):
    RA:  217.102347° = 14h28m24.56321512s
    Dec: 71.745251° = 71d44m42.90207861s
  Field 7 (drift_ra14h28m38s):
    RA:  217.156497° = 14h28m37.55923264s
    Dec: 71.745176° = 71d44